# Error Propagation in Python
## A Jupyter Notebook for Error Analysis

This notebook demonstrates error propagation using numerical methods.
We'll calculate how uncertainties in measurements combine to affect 
the final result of any calculation.

**Key concepts:**
- Total differential approximation
- Worst-case vs. statistical error combination
- Numerical derivatives using finite differences

## Theory

For a function f(x,y,z,...), the total differential gives us:

**δf = (∂f/∂x)δx + (∂f/∂y)δy + (∂f/∂z)δz + ...**

We combine individual error contributions in two ways:
1. **Worst-case error**: Σ|∂f/∂xi|δxi (assumes all errors align unfavorably)
2. **RSS error**: √(Σ(∂f/∂xi)²(δxi)²) (assumes independent random errors)

The RSS method is more realistic for most scientific measurements.

## Code Implementation

In [ ]:
import numpy as np
from math import log10, floor
import matplotlib.pyplot as plt

def round_to_sig_figs(x, sig_figs):
    """Round a number to specified significant figures"""
    if x == 0:
        return 0
    return round(x, -int(floor(log10(abs(x)))) + (sig_figs - 1))

def get_errors(Q, dQ, epsQ, fcn):
    """
    Calculate error propagation using finite differences
    
    Parameters:
    -----------
    Q : array-like
        Variables [x, y, z, ...]
    dQ : array-like  
        Uncertainties in each variable [δx, δy, δz, ...]
    epsQ : array-like
        Step sizes for finite difference approximation
    fcn : function
        Function to analyze f(Q)
    
    Returns:
    --------
    errors : list
        [worst_case_error, rss_error]
    """
    
    # Input validation
    Q = np.array(Q)
    dQ = np.array(dQ)
    epsQ = np.array(epsQ)
    
    if np.any(dQ <= 0):
        raise ValueError('Uncertainties must be positive')
    if len(Q) != len(dQ):
        raise ValueError('Q and dQ must have same length')
    if len(Q) != len(epsQ):
        raise ValueError('Q and epsQ must have same length')
    
    # Calculate partial derivatives using central differences
    n = len(Q)
    partial_derivatives = np.zeros(n)
    
    for i in range(n):
        h = epsQ[i]
        # Perturb ith variable
        Q_hi = Q.copy()
        Q_hi[i] += h
        Q_lo = Q.copy() 
        Q_lo[i] -= h
        # Central difference formula
        partial_derivatives[i] = (fcn(Q_hi) - fcn(Q_lo)) / (2*h)
    
    # Calculate individual error contributions
    error_contributions = partial_derivatives * dQ
    
    # Combine errors
    worst_case_error = np.sum(np.abs(error_contributions))
    rss_error = np.linalg.norm(error_contributions)
    
    return [worst_case_error, rss_error], partial_derivatives, error_contributions

## Example 1: Density Calculation

Let's calculate the density ρ = m/V where:
- mass m = 2.5 ± 0.02 g
- volume V = 0.25 ± 0.002 L

In [ ]:
# Define variables
m = 2.5     # mass in g
dm = 0.02   # uncertainty in g
V = 0.25    # volume in L
dV = 0.002  # uncertainty in L

# Create arrays
Q = np.array([m, V])
dQ = np.array([dm, dV])
epsQ = 1e-6 * np.abs(Q) + 1e-12  # Step sizes for numerical derivatives

# Define the density function
density = lambda Q: Q[0] / Q[1]  # ρ = m/V

# Calculate errors
errors, derivatives, contributions = get_errors(Q, dQ, epsQ, density)

# Round to 2 significant figures
errors_rounded = [round_to_sig_figs(err, 2) for err in errors]

# Display results
print("=== DENSITY CALCULATION ===")
print(f"Variables: m = {m} ± {dm} g, V = {V} ± {dV} L")
print(f"Function: ρ = m/V")
print(f"")
print(f"Density = {density(Q):.3f} g/L")
print(f"")
print(f"Partial derivatives:")
print(f"∂ρ/∂m = {derivatives[0]:.3f}")
print(f"∂ρ/∂V = {derivatives[1]:.3f}")
print(f"")
print(f"Error contributions:")
print(f"Mass contribution: ±{abs(contributions[0]):.4f}")
print(f"Volume contribution: ±{abs(contributions[1]):.4f}")
print(f"")
print(f"Combined errors:")
print(f"Worst-case error: ±{errors_rounded[0]} g/L")
print(f"RSS error: ±{errors_rounded[1]} g/L")
print(f"")
print(f"Final result (worst-case): {density(Q):.3f} ± {errors_rounded[0]} g/L")

## Example 2: Kinetic Energy

Calculate kinetic energy KE = ½mv² where:
- mass m = 1.0 ± 0.1 kg
- velocity v = 10.0 ± 2.0 m/s

In [ ]:
# Define variables
m_ke = 1.0    # mass in kg
dm_ke = 0.1   # uncertainty
v = 10.0      # velocity in m/s
dv = 2.0      # uncertainty

Q_ke = np.array([m_ke, v])
dQ_ke = np.array([dm_ke, dv])
epsQ_ke = 1e-6 * np.abs(Q_ke) + 1e-12

# Define kinetic energy function
kinetic_energy = lambda Q: 0.5 * Q[0] * Q[1]**2

# Calculate errors
errors_ke, derivatives_ke, contributions_ke = get_errors(Q_ke, dQ_ke, epsQ_ke, kinetic_energy)
errors_ke_rounded = [round_to_sig_figs(err, 2) for err in errors_ke]

print("=== KINETIC ENERGY CALCULATION ===")
print(f"Variables: m = {m_ke} ± {dm_ke} kg, v = {v} ± {dv} m/s")
print(f"Function: KE = ½mv²")
print(f"")
print(f"Kinetic Energy = {kinetic_energy(Q_ke):.1f} J")
print(f"")
print(f"Partial derivatives:")
print(f"∂KE/∂m = {derivatives_ke[0]:.1f}")
print(f"∂KE/∂v = {derivatives_ke[1]:.1f}")
print(f"")
print(f"Error contributions:")
print(f"Mass contribution: ±{abs(contributions_ke[0]):.1f} J")
print(f"Velocity contribution: ±{abs(contributions_ke[1]):.1f} J")
print(f"")
print(f"Final result (RSS): {kinetic_energy(Q_ke):.1f} ± {errors_ke_rounded[1]} J")

## Visualization: Error Contributions

In [ ]:
# Create a bar chart showing error contributions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Density example
variables_density = ['Mass (m)', 'Volume (V)']
ax1.bar(variables_density, np.abs(contributions), alpha=0.7)
ax1.set_ylabel('Error Contribution')
ax1.set_title('Error Contributions to Density')
ax1.grid(True, alpha=0.3)

# Kinetic energy example  
variables_ke = ['Mass (m)', 'Velocity (v)']
ax2.bar(variables_ke, np.abs(contributions_ke), alpha=0.7, color='orange')
ax2.set_ylabel('Error Contribution (J)')
ax2.set_title('Error Contributions to Kinetic Energy')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Student Exercises

Try these exercises by modifying the code above:

1. **Gas Law**: Calculate pressure using P = nRT/V
   - n = 2.0 ± 0.1 mol
   - R = 8.314 J/(mol·K) (exact)
   - T = 300 ± 5 K
   - V = 0.025 ± 0.001 m³

2. **Power**: Calculate P = V²/R
   - V = 12.0 ± 0.5 V
   - R = 100 ± 5 Ω

3. **Pendulum Period**: T = 2π√(L/g)
   - L = 1.0 ± 0.02 m
   - g = 9.81 ± 0.01 m/s²

4. **Three Variables**: Try f(x,y,z) = xyz/(x+y+z)
   - x = 2.0 ± 0.1
   - y = 3.0 ± 0.2  
   - z = 4.0 ± 0.3

In [ ]:
# Template for exercises:
def solve_exercise(variables, uncertainties, function, function_name):
    """Template function for student exercises"""
    Q = np.array(variables)
    dQ = np.array(uncertainties)  
    epsQ = 1e-6 * np.abs(Q) + 1e-12
    
    errors, derivatives, contributions = get_errors(Q, dQ, epsQ, function)
    errors_rounded = [round_to_sig_figs(err, 2) for err in errors]
    
    print(f"=== {function_name.upper()} ===")
    print(f"Result = {function(Q):.3f}")
    print(f"RSS Error = ±{errors_rounded[1]:.3f}")
    print(f"Final answer: {function(Q):.3f} ± {errors_rounded[1]:.3f}")
    
    return errors, derivatives, contributions

# Example usage:
# gas_law = lambda Q: (Q[0] * 8.314 * Q[1]) / Q[2]  # P = nRT/V
# solve_exercise([2.0, 300, 0.025], [0.1, 5, 0.001], gas_law, "Gas Law Pressure")

print("Ready for exercises! Use the template above to solve the problems.")